In [ ]:
!pip install spacy pdfplumber nltk
!python -m spacy download en_core_web_sm
import nltk
nltk.download('punkt')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 47.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 39.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 83.8 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
from google.colab import files
import pdfplumber
uploaded = files.upload()
def extract_text_from_pdf(pdf_path):
    with pdfplumber.open(pdf_path) as pdf:
        return "\n".join([page.extract_text() or "" for page in pdf.pages])
def extract_text_from_txt(txt_file):
    return open(txt_file, "r", encoding="utf-8").read()

Saving Resume2_merged.pdf to Resume2_merged.pdf


In [ ]:
file_name = next(iter(uploaded))
if file_name.endswith(".pdf"):
    raw_text = extract_text_from_pdf(file_name)
else:
    raw_text = extract_text_from_txt(file_name)
print(raw_text[:1000])

Rosie Miller
Pittsburgh, PA 15201
(555) 555-5555
example@example.com
PROFESSIONAL SUMMARY
Experienced Restaurant Manager bringing demonstrated success in developing and motivating strong restaurant teams capable of
handling over 200 customers hourly. Keeps all areas clean and sanitized while managing inventory and preventing waste. Consistent
career history of operations improvement, team building and revenue increases.
WORK HISTORY
March 2014 to Current
Outback Steakhouse – Pittsburgh, PA
Restaurant Manager
Reduced labor costs by 17% percent while maintaining excellent service and profit levels
Managed a 7-person team of cooks and back of house staff and a team of 8 front house staff for a busy steakhouse restaurant
Continuously evaluated business operations to effectively align workflows for optimal area coverage increasing customer satisfaction
rating by 80%
December 2010 to February 2014
TGI Fridays – Pittsburgh, PA Assistant
Restaurant Manager
Reduced restaurant’s annual food and 

In [ ]:
import spacy
from spacy.matcher import PhraseMatcher
nlp = spacy.load("en_core_web_sm")
doc = nlp(raw_text)
SKILLS = [
    "python", "java", "c++", "sql", "html", "css", "javascript",
    "machine learning", "deep learning", "nlp", "ui/ux design",
    "version control", "seo", "typography", "color theory",
    "web accessibility", "front-end", "web design tools", "business operations",
    "inventory control", "record keeping", "marketing", "advertising",
    "conflict resolution", "staff management", "performance improvement"
]
DEGREES = [
    "bachelor of arts", "bachelor of science", "bachelor of technology",
    "master of science", "master of technology", "master of design",
    "master of it management", "hospitality management",
    "b.tech", "m.tech", "b.sc", "m.sc", "b.e", "m.e"
]

def get_matches(text, keywords):
    matcher = PhraseMatcher(nlp.vocab, attr="LOWER")
    patterns = [nlp.make_doc(keyword.lower()) for keyword in keywords]
    matcher.add("TAGS", patterns)
    doc = nlp(text.lower())
    matches = matcher(doc)
    return list(set([doc[start:end].text for match_id, start, end in matches]))

In [ ]:
def extract_entities(text):
    doc = nlp(text)
    result = {
        "Name": "",
        "Skills": [],
        "Degree": [],
        "Institutions": [],
        "Experience": []
    }

    for ent in doc.ents:
        if ent.label_ == "PERSON":
            result["Name"] = ent.text
            break

    result["Institutions"] = list(set([ent.text for ent in doc.ents if ent.label_ == "ORG"]))
    result["Experience"] = [sent.text for sent in doc.sents if any(keyword in sent.text.lower() for keyword in ["experience", "manager", "designer", "leader"])]
    result["Skills"] = get_matches(text, SKILLS)
    result["Degree"] = get_matches(text, DEGREES)
    return result
structured_data = extract_entities(raw_text)
structured_data


{'Name': 'Rosie Miller',
 'Skills': ['typography',
  'web design tools',
  'inventory control',
  'business operations',
  'version control',
  'record keeping',
  'staff management',
  'marketing',
  'front-end',
  'color theory',
  'conflict resolution',
  'advertising',
  'web accessibility',
  'ui/ux design',
  'performance improvement',
  'seo'],
 'Degree': ['hospitality management',
  'bachelor of arts',
  'master of it management'],
 'Institutions': ['nec',
  'BJ’s Restaurants, Inc – Pittsburgh',
  'Park Point University',
  'Coordinated',
  'HRD',
  'Current\nOutback Steakhouse'],
 'Experience': ['Rosie Miller\nPittsburgh, PA 15201\n(555) 555-5555\nexample@example.com\nPROFESSIONAL SUMMARY\nExperienced Restaurant Manager bringing demonstrated success in developing and motivating strong restaurant teams capable of\nhandling over 200 customers hourly.',
  'WORK HISTORY\nMarch 2014 to Current\nOutback Steakhouse – Pittsburgh, PA\nRestaurant Manager\nReduced labor costs by 17% perc

In [ ]:
import json
with open("parsed_resume.json", "w") as f:
    json.dump(structured_data, f, indent=2)
files.download("parsed_resume.json")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>